# 04 - Filter

In [1]:
# imports
import pandas as pd
import pickle
import sqlite3

from sqlalchemy import create_engine

In [27]:
rec_df = pickle.load(open('../Data/filter_rec_df.pkl', 'rb'))

In [28]:
rec_df.head()

,Static Orb,Sensory Deprivation,Road of Return,Storm Crow,Walking Sponge,Ravnica at War,Torrent of Fire,Wyluli Wolf,Pteramander,Nantuko Elder,...,legalities_vintage,legalities_penny,legalities_brawl,legalities_modern,legalities_oldschool,legalities_pioneer,legalities_legacy,legalities_commander,cmc,card_link
name,,,,,,,,,,,,,,,,,,,,,
Static Orb,0.000,0.873,0.923,0.902,0.904,0.777,0.903,0.903,0.937,0.835,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,https://scryfall.com/card/7ed/319/static-orb?u...
Sensory Deprivation,0.873,0.000,0.922,0.740,0.756,0.743,0.899,0.704,0.801,0.732,...,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,https://scryfall.com/card/m14/71/sensory-depri...
Road of Return,0.923,0.922,0.000,0.955,0.883,0.887,0.881,0.907,0.882,0.868,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,https://scryfall.com/card/c19/34/road-of-retur...
Storm Crow,0.902,0.740,0.955,0.000,0.785,0.856,0.929,0.831,0.828,0.778,...,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,2.0,https://scryfall.com/card/9ed/100/storm-crow?u...
Walking Sponge,0.904,0.756,0.883,0.785,0.000,0.886,0.880,0.492,0.807,0.653,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,https://scryfall.com/card/ulg/47/walking-spong...


In [4]:
# connect to SQL database
conn = sqlite3.connect('../Data/MTG_Recommender.db')

In [23]:
sql = """
SELECT * FROM filter
LIMIT 5
"""

list(pd.read_sql_query(sql, conn).columns)

['name',
 'colors_B',
 'color_identity_B',
 'colors_G',
 'color_identity_G',
 'colors_R',
 'color_identity_R',
 'colors_U',
 'color_identity_U',
 'colors_W',
 'color_identity_W',
 'card_type_Creature',
 'card_type_Instant',
 'card_type_Enchantment',
 'card_type_Sorcery',
 'card_type_Artifact',
 'card_type_Land',
 'card_type_Planeswalker',
 'card_type_Tribal',
 'sub_type_Pegasus',
 'sub_type_Kraken',
 'sub_type_Satyr',
 'sub_type_Drone',
 'sub_type_Kor',
 'sub_type_Slug',
 'sub_type_Hippogriff',
 'sub_type_Masticore',
 'sub_type_Hyena',
 'sub_type_Homarid',
 'sub_type_Antelope',
 'sub_type_Rabbit',
 'sub_type_Wolf',
 'sub_type_Goblin',
 'sub_type_Garruk',
 'sub_type_Mystic',
 'sub_type_Hag',
 'sub_type_Vedalken',
 'sub_type_Scarecrow',
 'sub_type_Barbarian',
 'sub_type_Camel',
 'sub_type_Wurm',
 'sub_type_Kasmina',
 'sub_type_Fortification',
 'sub_type_Wombat',
 'sub_type_Rebel',
 'sub_type_Leech',
 'sub_type_Illusion',
 'sub_type_Surrakar',
 'sub_type_Bolas',
 'sub_type_Hippo',
 'sub_t

In [38]:
u_sub_type = 'Human'
u_color = 'R'
u_color_id = 'G'
u_card_type = 'Creature'
u_cmc_lower = 0
u_cmc_upper = 4
u_legalities = 'modern'

sql = """
SELECT name FROM filter
WHERE 
"""
sql += f"colors_{u_color} != 0 "
sql += f"AND cmc BETWEEN {u_cmc_lower} AND {u_cmc_upper} "
sql += f"AND sub_type_{u_sub_type} != 0 "


filtered_list = list(pd.read_sql_query(sql, conn)['name'].values)
filtered_list

['Keldon Raider',
 'Soul-Scar Mage',
 'Planebound Accomplice',
 'Arc Mage',
 'Akroan Line Breaker',
 'Reckless Cohort',
 'Embraal Gear-Smasher',
 'Seasoned Pyromancer',
 'Blood Cultist',
 'Burning Shield Askari',
 'Nef-Crop Entangler',
 'Kolaghan Stormsinger',
 'Kolaghan Forerunners',
 'Starke of Rath',
 'League Guildmage',
 'Najeela, the Blade-Blossom',
 'Deputized Protester',
 'Agent of Stromgald',
 'Abbot of Keral Keep',
 'Keldon Vandals',
 'Tuknir Deathlock',
 'Flamewright',
 'Stormfist Crusader',
 'Lyzolda, the Blood Witch',
 'Vulshok Heartstoker',
 'Fireslinger',
 'Paragon of Fierce Defiance',
 'Valduk, Keeper of the Flame',
 'Captivating Crew',
 'Forerunner of the Empire',
 'Borderland Marauder',
 'Sandstone Warrior',
 'Barbarian Lunatic',
 'Brazen Freebooter',
 'Bola Warrior',
 'Fresh-Faced Recruit',
 'Chainer, Nightmare Adept',
 'Humble Defector',
 'Keldon Champion',
 'Harsh Mentor',
 'Stormcaller of Keranos',
 'Marchesa, the Black Rose',
 'Valakut Invoker',
 'Gnat Alley Creep

In [32]:
rec_df[filtered_list].T['Shock'].sort_values()[0:10]

Shock Troops                                0.247
Skirsdag Cultist                            0.311
Grim Lavamancer                             0.365
Fearless Halberdier                         0.428
Valakut Invoker                             0.433
Labyrinth Champion                          0.443
Sell-Sword Brute                            0.445
Vulshok Sorcerer                            0.452
Embereth Shieldbreaker // Battle Display    0.457
Prodigal Pyromancer                         0.459
Name: Shock, dtype: float64